In [26]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2

In [2]:
!ls

classic.ipynb  donuts-found.jpg  prepare-data-for-yolo.ipynb
data.csv       img		 test.png
detect	       img.zip		 Untitled.ipynb


In [3]:
df = pd.read_csv('data.csv')

In [4]:
df

,img_id,cam_id,seq_id,frame_id,bbox_id,x_min,x_max,y_min,y_max,label_l1,label_l2,fao_mfa,train,val,test
0,da7c5ef0-db28-11ea-b693-9b3523cc10a5,60,13209,33,1,626,754,-1,88,Human,HUMAN,81,True,False,False
1,da7c5ef0-db28-11ea-b693-9b3523cc10a5,60,13209,33,3,544,710,15,203,Human,HUMAN,81,True,False,False
2,da7c5ef0-db28-11ea-b693-9b3523cc10a5,60,13209,33,4,629,735,210,562,Human,HUMAN,81,True,False,False
3,da7c5ef0-db28-11ea-b693-9b3523cc10a5,60,13209,33,5,615,883,550,907,Human,HUMAN,81,True,False,False
4,da7c5ef0-db28-11ea-b693-9b3523cc10a5,60,13209,33,6,885,962,826,875,Unknown,OTH,81,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549204,fcdecc18-5952-11ec-a87d-47788f674eed,93,15,28,126812,227,510,702,992,Yellowfin tuna,YFT,81,False,False,True
549205,fcdeef2c-5952-11ec-a8ae-cf526681fdda,93,15,77,128339,223,518,700,996,Yellowfin tuna,YFT,81,False,False,True
549206,fcdeef2c-5952-11ec-a8ae-cf526681fdda,93,15,77,128338,875,1127,668,1030,Yellowfin tuna,YFT,81,False,False,True
549207,fcdeef2c-5952-11ec-a8ae-cf526681fdda,93,15,77,128337,1069,1465,487,674,Yellowfin tuna,YFT,81,False,False,True


In [5]:
df['img_id'][0]

'da7c5ef0-db28-11ea-b693-9b3523cc10a5'

In [6]:
df = df.drop(['cam_id', 'seq_id', 'frame_id', 'bbox_id', 'label_l2', 'fao_mfa'],axis=1)

In [7]:
df 

,img_id,x_min,x_max,y_min,y_max,label_l1,train,val,test
0,da7c5ef0-db28-11ea-b693-9b3523cc10a5,626,754,-1,88,Human,True,False,False
1,da7c5ef0-db28-11ea-b693-9b3523cc10a5,544,710,15,203,Human,True,False,False
2,da7c5ef0-db28-11ea-b693-9b3523cc10a5,629,735,210,562,Human,True,False,False
3,da7c5ef0-db28-11ea-b693-9b3523cc10a5,615,883,550,907,Human,True,False,False
4,da7c5ef0-db28-11ea-b693-9b3523cc10a5,885,962,826,875,Unknown,True,False,False
...,...,...,...,...,...,...,...,...,...
549204,fcdecc18-5952-11ec-a87d-47788f674eed,227,510,702,992,Yellowfin tuna,False,False,True
549205,fcdeef2c-5952-11ec-a8ae-cf526681fdda,223,518,700,996,Yellowfin tuna,False,False,True
549206,fcdeef2c-5952-11ec-a8ae-cf526681fdda,875,1127,668,1030,Yellowfin tuna,False,False,True
549207,fcdeef2c-5952-11ec-a8ae-cf526681fdda,1069,1465,487,674,Yellowfin tuna,False,False,True


In [13]:
image_path = 'img'

In [14]:
from PIL import Image

In [15]:
test_img = Image.open(f'{image_path}/da7c5ef0-db28-11ea-b693-9b3523cc10a5.jpg')
test_img.size

(1920, 1030)

In [16]:
first_image_bboxes = df[df['img_id']=='da7c5ef0-db28-11ea-b693-9b3523cc10a5']
first_image_bboxes

,img_id,x_min,x_max,y_min,y_max,label_l1,train,val,test
0,da7c5ef0-db28-11ea-b693-9b3523cc10a5,626,754,-1,88,Human,True,False,False
1,da7c5ef0-db28-11ea-b693-9b3523cc10a5,544,710,15,203,Human,True,False,False
2,da7c5ef0-db28-11ea-b693-9b3523cc10a5,629,735,210,562,Human,True,False,False
3,da7c5ef0-db28-11ea-b693-9b3523cc10a5,615,883,550,907,Human,True,False,False
4,da7c5ef0-db28-11ea-b693-9b3523cc10a5,885,962,826,875,Unknown,True,False,False


In [19]:
df_train = df[df['train']]
df_val = df[df['val']]
df_test =  df[df['test']]
len(df_train),len(df_test),len(df_val)

(390171, 80093, 78945)

In [20]:
df_train.columns

Index(['img_id', 'x_min', 'x_max', 'y_min', 'y_max', 'label_l1', 'train',
       'val', 'test'],
      dtype='object')

In [21]:
map_ids = {name: num for num, name in enumerate(set(df['label_l1']))}

In [24]:
from tqdm import tqdm

In [28]:
from shutil import copy

In [29]:
os.makedirs('annotations', exist_ok=True)
os.makedirs('annotations/images', exist_ok=True)
os.makedirs('annotations/images/train', exist_ok=True)
os.makedirs('annotations/images/val', exist_ok=True)
os.makedirs('annotations/images/test', exist_ok=True)
os.makedirs('annotations/labels', exist_ok=True)
os.makedirs('annotations/labels/train', exist_ok=True)
os.makedirs('annotations/labels/val', exist_ok=True)
os.makedirs('annotations/labels/test', exist_ok=True)

for idx, row in tqdm(df_train.iterrows(),total=len(df_train)):
    img_path = f"{image_path}/{row['img_id']}.jpg"
    
  
    img = cv2.imread(img_path)
    h, w, _ = img.shape  
    x_center = (row['x_min'] + row['x_max']) / 2 / w
    y_center = (row['y_min'] + row['y_max']) / 2 / h
    bbox_width = (row['x_max'] - row['x_min']) / w
    bbox_height = (row['y_max'] - row['y_min']) / h

    class_id = map_ids[row['label_l1']]
    
    txt_file_path = os.path.join('annotations/labels/train', os.path.splitext(os.path.basename(img_path))[0] + '.txt')
    img_file_path =  os.path.join('annotations/images/train', os.path.splitext(os.path.basename(img_path))[0] + '.jpg')
    with open(txt_file_path, 'a') as f:
        f.write(f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}\n")
    copy(img_path,img_file_path)



100%|████████████████████████████████████████████████████████████| 390171/390171 [28:23<00:00, 229.04it/s]


In [30]:
for idx, row in tqdm(df_test.iterrows(),total=len(df_test)):
    img_path = f"{image_path}/{row['img_id']}.jpg"
    
    img = cv2.imread(img_path)
    h, w, _ = img.shape  

    x_center = (row['x_min'] + row['x_max']) / 2 / w
    y_center = (row['y_min'] + row['y_max']) / 2 / h
    bbox_width = (row['x_max'] - row['x_min']) / w
    bbox_height = (row['y_max'] - row['y_min']) / h

    class_id = map_ids[row['label_l1']]
    
    txt_file_path = os.path.join('annotations/labels/test', os.path.splitext(os.path.basename(img_path))[0] + '.txt')
    img_file_path =  os.path.join('annotations/images/test', os.path.splitext(os.path.basename(img_path))[0] + '.jpg')
    with open(txt_file_path, 'a') as f:
        f.write(f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}\n")
    copy(img_path,img_file_path)


100%|██████████████████████████████████████████████████████████████| 80093/80093 [05:11<00:00, 257.15it/s]


In [31]:
for idx, row in tqdm(df_val.iterrows(),total=len(df_val)):
    img_path = f"{image_path}/{row['img_id']}.jpg"
    
    img = cv2.imread(img_path)
    h, w, _ = img.shape  

    x_center = (row['x_min'] + row['x_max']) / 2 / w
    y_center = (row['y_min'] + row['y_max']) / 2 / h
    bbox_width = (row['x_max'] - row['x_min']) / w
    bbox_height = (row['y_max'] - row['y_min']) / h

    class_id = map_ids[row['label_l1']]
    
    txt_file_path = os.path.join('annotations/labels/val', os.path.splitext(os.path.basename(img_path))[0] + '.txt')
    img_file_path =  os.path.join('annotations/images/val', os.path.splitext(os.path.basename(img_path))[0] + '.jpg')
    with open(txt_file_path, 'a') as f:
        f.write(f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}\n")
    copy(img_path,img_file_path)


100%|██████████████████████████████████████████████████████████████| 78945/78945 [05:38<00:00, 233.30it/s]


In [32]:
import json
with open(txt_file_path, 'w') as f:
    json.dump(map_ids,f)

In [33]:
map_ids

{'Rainbow runner': 0,
 'Snake mackerel': 1,
 'Shortbill spearfish': 2,
 'Yellowfin tuna': 3,
 'Tuna': 4,
 'Skipjack tuna': 5,
 'Black marlin': 6,
 'Escolar': 7,
 'Thresher shark': 8,
 'Pomfret': 9,
 'Shark': 10,
 'Roudie scolar': 11,
 'Human': 12,
 'Long snouted lancetfish': 13,
 'Brama': 14,
 'Sickle pomfret': 15,
 'Mahi mahi': 16,
 'Blue marlin': 17,
 'Pelagic stingray': 18,
 'Wahoo': 19,
 'Striped marlin': 20,
 'Opah': 21,
 'Indo Pacific sailfish': 22,
 'Mola mola': 23,
 'Bigeye tuna': 24,
 'Swordfish': 25,
 'Marlin': 26,
 'Great barracuda': 27,
 'Albacore': 28,
 'Lancetfish': 29,
 'Oilfish': 30,
 'Water': 31,
 'No fish': 32,
 'Unknown': 33}